# Relativistic orthogonalization: Adversarial examples
Find adversarial examples that get at when orthogonalization borks, and when it borks even under arbitrary linear combinations of the input vectors.

## Author:
- **David W. Hogg** (NYU)

## Notes:

In [ ]:
import numpy as np
NDIGIT = 10 # rounding / precision limit we care about

In [ ]:
def metric(s, d):
    foo = np.ones(d + s)
    foo[s:] = -1.
    return np.diag(foo)

def inner(u, v):
    return u.T @ METRIC @ v

def orthogonalize(vs):
    dpluss, _ = METRIC.shape
    n, dps = vs.shape
    assert n <= dpluss
    assert dps == dpluss
    us = np.zeros_like(vs)
    for i in range(n):
        ui = 1. * vs[i]
        for j in range(i):
            ui -= (inner(vs[i], us[j]) / inner(us[j], us[j])) * us[j]
        us[i] = ui
    return us

In [ ]:
# set up an extremely weird kind of Lorentz space
s, d = 2, 4 # two times!
METRIC = metric(s, d)
print(METRIC)

In [ ]:
# make adversarial examples by hand
a1 = np.array([4.8, 0.0, 3.0, 4.0, 0.0, 0.0])
a2 = np.array([0.4, 0.0, 2.0,-2.0, 0.0, 0.0])
a3 = np.array([0.0, 4.8, 0.0, 0.0, 3.0, 4.0])
a4 = np.array([0.0, 0.4, 0.0, 0.0, 2.0,-2.0])
print([np.round(inner(a, a), NDIGIT) for a in [a1, a2, a3, a4]])

In [ ]:
# this is bad
us = orthogonalize(np.array([a1, a2]))
print([np.round(inner(u, u), NDIGIT) for u in us])
us = orthogonalize(np.array([a3, a4]))
print([np.round(inner(u, u), NDIGIT) for u in us])

In [ ]:
# perhaps suprisingly, this is also bad
us = orthogonalize(np.array([a2, a1]))
print([np.round(inner(u, u), NDIGIT) for u in us])
us = orthogonalize(np.array([a4, a3]))
print([np.round(inner(u, u), NDIGIT) for u in us])

In [ ]:
# even linear combinations of the bad pairs bork!
rng = np.random.default_rng(17)
TINY = 10 ** -NDIGIT
ntrial = 2 ** 3
nsuccess = 0
for t in range(ntrial):
    ras = rng.normal(size=(2, 2)) @ np.array([a1, a2])
    us = orthogonalize(ras)
    nsuccess += 1
    if np.abs(np.prod([inner(u, u) for u in us])) < TINY:
        print(t, "failed:", [np.round(inner(u, u), NDIGIT) for u in us])
        nsuccess -= 1
print("Out of", ntrial, "trials:", nsuccess, "successes")

In [ ]:
# it even borks if we do linear combinations of the pair of pairs; ie, the outer product of the subspaces
rng = np.random.default_rng(17)
ntrial = 2 ** 3
nsuccess = 0
for t in range(ntrial):
    ras = rng.normal(size=(4, 4)) @ np.array([a1, a2, a3, a4])
    us = orthogonalize(ras)
    nsuccess += 1
    if np.abs(np.prod([inner(u, u) for u in us])) < TINY:
        print(t, "failed:", [np.round(inner(u, u), NDIGIT) for u in us])
        nsuccess -= 1
print("Out of", ntrial, "trials:", nsuccess, "successes")

In [ ]:
# perhaps unsuprisingly, this is fine!
us = orthogonalize(np.array([a2, a3]))
print([np.round(inner(u, u), NDIGIT) for u in us])
us = orthogonalize(np.array([a1, a4]))
print([np.round(inner(u, u), NDIGIT) for u in us])
us = orthogonalize(np.array([a1, a3]))
print([np.round(inner(u, u), NDIGIT) for u in us])
us = orthogonalize(np.array([a2, a4]))
print([np.round(inner(u, u), NDIGIT) for u in us])

In [ ]:
# now what if the pairs aren't so separated? Then things are fine!
# that is, if the combination a1, a2, a3, a4 is not just the outer product of the a1, a2 and a3, a4 spaces.
a1 = np.array([4.8, 0.0, 3.0, 4.0, 0.0, 0.0])
a2 = np.array([0.4, 0.0, 2.0,-2.0, 0.0, 0.0])
a3 = np.array([0.0, 4.8, 0.0, 3.0, 4.0, 0.0])
a4 = np.array([0.0, 0.4, 0.0, 2.0,-2.0, 0.0])
print([np.round(inner(a, a), NDIGIT) for a in [a1, a2, a3, a4]])
rng = np.random.default_rng(17)
ntrial = 2 ** 10
nsuccess = 0
for t in range(ntrial):
    ras = rng.normal(size=(4, 4)) @ np.array([a1, a2, a3, a4])
    us = orthogonalize(ras)
    nsuccess += 1
    if np.abs(np.prod([inner(u, u) for u in us])) < TINY:
        print(t, "failed:", [np.round(inner(u, u), NDIGIT) for u in us])
        nsuccess -= 1
print("Out of", ntrial, "trials:", nsuccess, "successes")

In [ ]:
# what if we just add one more random vector?
rng = np.random.default_rng(17)
a3 = rng.normal(size=(d + s))
print([np.round(inner(a, a), NDIGIT) for a in [a1, a2, a3]])
ntrial = 2 ** 10
nsuccess = 0
for t in range(ntrial):
    ras = rng.normal(size=(3, 3)) @ np.array([a1, a2, a3])
    us = orthogonalize(ras)
    nsuccess += 1
    if np.abs(np.prod([inner(u, u) for u in us])) < TINY:
        print(t, "failed:", [np.round(inner(u, u), NDIGIT) for u in us])
        nsuccess -= 1
print("Out of", ntrial, "trials:", nsuccess, "successes")

In [ ]:
# and yet: You never hit nulls randomly
rng = np.random.default_rng(17)
ntrial = 2 ** 16
nsuccess = 0
for t in range(ntrial):
    ras = rng.normal(size=(2, d + s))
    us = orthogonalize(ras)
    nsuccess += 1
    if np.abs(np.prod([inner(u, u) for u in us])) < TINY:
        print(t, "failed on ", ras, us, [inner(u, u) for u in us])
        nsuccess -= 1
print("Out of", ntrial, "trials:", nsuccess, "successes")